# Chain of thought

## Step 1: Setup and Library Imports

In [1]:
import json
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from groq import Groq  # Assuming you have access to Groq's client
import os

def load_json_data(json_path):
    with open(json_path, 'r', encoding='utf-8') as file:
        return json.load(file)

def create_tfidf_vectors(data):
    documents = [entry['question'] + " " + entry['code'] for entry in data]  # Context includes both code and the question
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(documents)
    return vectorizer, tfidf_matrix


In [10]:
from dotenv import load_dotenv

load_dotenv()
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

## Step 2: Implementing Document Retrieval

In [11]:
def retrieve_documents(query, data, vectorizer, tfidf_matrix):
    query_vec = vectorizer.transform([query])
    similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()
    top_n_indices = np.argsort(-similarities)[:5]  # Retrieves the top 5 documents
    return [data[i] for i in top_n_indices if similarities[i] > 0]


## Step 3: Generating Responses with Chain of Thought

In [12]:
def generate_response_with_chain_of_thought(query, documents):
    client = Groq(api_key=os.environ.get("GROQ_API_KEY"))
    # Construct a reasoning path from the documents
    reasoning = "Based on the provided code and context, "
    context = " ".join([f"{doc['question']} Hence, {doc['answer']}." for doc in documents])
    reasoning += context
    combined_input = f"Question: {query} {reasoning} Therefore, "
    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": combined_input,
            }
        ],
        model="llama-3.3-70b-versatile",
    )
    return response.choices[0].message.content


## Main Execution

In [13]:
if __name__ == "__main__":
    json_path = 'Dataset/output_data.json'
    data = load_json_data(json_path)
    vectorizer, tfidf_matrix = create_tfidf_vectors(data)
    
    user_query = """def Make Suite From Dict d label None suite Suite label label suite Set Dict d suite Normalize return suite
 What does the code make?"""
    relevant_documents = retrieve_documents(user_query, data, vectorizer, tfidf_matrix)
    
    if not relevant_documents:
        print("No relevant documents found.")
    else:
        response = generate_response_with_chain_of_thought(user_query, relevant_documents)
        print(response)


The code appears to be creating a test suite from a dictionary. 

Here's a breakdown of what the code does:

1. It defines a function `MakeSuiteFromDict` that takes in a dictionary `d`, a label, and a suite.
2. The function iterates over the dictionary `d` and creates a test suite with the given label.
3. The function normalizes the test suite and returns it.

Given the context, it seems that the code is used to create a test suite for Flask tests. 

A test suite typically has a collection of test cases, so in this case, the test suite would have all the Flask tests.

Therefore, the code makes a test suite that contains all the Flask tests. 

Here is a simple representation of what the code might look like:
```
def MakeSuiteFromDict(d, label=None, suite=None):
    if suite is None:
        suite = unittest.TestSuite()
    for key, value in d.items():
        if isinstance(value, unittest.TestCase):
            suite.addTest(value)
        elif isinstance(value, dict):
            suite